# Keras MLP -> best Log Loss 0.555
## Overview ##
This kernel uses seeding and relative seeding with an MLP network. Spread on loss was very small, so kfold not necessary

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from time import localtime
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, BatchNormalization
from keras import regularizers
from keras.callbacks import EarlyStopping, TensorBoard, ProgbarLogger
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold




# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../NCAA"]).decode("utf8"))

# Any results you write to the current directory are saved as output.


Using TensorFlow backend.


Data
Forests_Rank.ipynb
GiantKillerInfo.txt
KaggleKernelwSeeds.ipynb
KerasBinaryClassifier_Intro.ipynb
KerasMLP_only2003.csv
KerasMLP_onlyBPI_2009-2013.csv
KerasMLP_onlyseed.csv
KerasMLP_onlyseeds.csv
KerasMLP_RankAndSeeds.ipynb
KerasMLP_Rank_K20.ipynb
KerasMLP_Rank_Kfold.ipynb
KerasMLP_RankSeedsYear_1_2.csv
KerasMLP_RankSeedsYear_1.csv
KerasMLP_RankSeedsYear.csv
KerasMLP_SeedsandYear_1.csv
KerasMLP_SeedsandYear.csv
KerasMLP_SeedsOnly.ipynb
KerasMLP_SeedswithTB.ipynb
logreg_withseedtoo.csv
logs
MasseyOrdinalsExp.ipynb
MLP_first_go.csv
MLP_with_seeds.csv
SKLearn - SVM.ipynb
sonar.csv
SVM_first_go.csv
SVM_Rank.ipynb
Tensorflow.ipynb



### Lock Random Seed

In [2]:
seed = 13
np.random.seed(seed)

### Load Training data


In [3]:
data_dir = '../NCAA/Data/'
df_seeds = pd.read_csv(data_dir + 'NCAATourneySeeds.csv')
df_tour = pd.read_csv(data_dir + 'NCAATourneyCompactResults.csv')
df_tour.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,136,1116,63,1234,54,N,0
1,1985,136,1120,59,1345,58,N,0
2,1985,136,1207,68,1250,43,N,0
3,1985,136,1229,58,1425,55,N,0
4,1985,136,1242,49,1325,38,N,0


Incorporate Massey Ordinals - MAS, SAG, POM

In [4]:
#Load Data
df_rank = pd.read_csv(data_dir+ 'MasseyOrdinals.csv')
#Choose Latest Ranking 
df_rank = df_rank[df_rank.RankingDayNum>=133]
df_rank = df_rank[df_rank.Season>=2010]

#Selectr rankings of interest and make each ranking system ranking into a separate column
df_merge = pd.merge(df_rank.loc[df_rank['SystemName']=='MAS'],
                    df_rank.loc[df_rank['SystemName']=='SAG'], how='left', 
                    on=['Season', 'TeamID', 'RankingDayNum'])
df_rank = pd.merge(left=df_merge, right=df_rank.loc[df_rank['SystemName']=='POM'], 
                  how='left', on=['Season', 'TeamID', 'RankingDayNum'] )

In [5]:
# Drop
df_rank.drop(labels=['SystemName_x', 'SystemName_y', 'SystemName','RankingDayNum'], inplace=True, axis =1)
df_rank.rename(columns={'OrdinalRank_x':'MAS', 'OrdinalRank_y':'SAG', 'OrdinalRank':'POM'}, inplace=True)

df_rank.head()

,Season,TeamID,MAS,SAG,POM
0,2010,1102,239,237,246
1,2010,1103,89,105,119
2,2010,1104,79,72,64
3,2010,1105,329,337,336
4,2010,1106,276,300,302


Cut seasons prior to Max Ranking date

In [6]:
df_seeds = df_seeds[df_seeds.Season>=min(df_rank.Season) ]
df_seeds = df_seeds[ df_seeds.Season<=max(df_rank.Season)]
df_tour = df_tour[df_tour.Season>=min(df_rank.Season)]
df_tour = df_tour[ df_tour.Season<=max(df_rank.Season)]
# df_tour.head()

Cut off the region identifier from the seed number

In [7]:
def seed_to_int(seed):
    #Get just the digits from the seeding. Return as int
    s_int = int(seed[1:3])
    return s_int
df_seeds['seed_int'] = df_seeds.Seed.apply(seed_to_int)
df_seeds.drop(columns=['Seed'], inplace=True) # This is the string label

In [8]:
df_tour.drop(labels=['DayNum', 'WScore', 'LScore','WLoc', 'NumOT'], inplace=True, axis=1)
# df_tour.head()

Merge the Seeds with their corresponding TeamIDs in the compact results dataframe.

In [9]:
df_winSeeds = df_seeds.rename(columns={'TeamID':'WTeamID', 'seed_int':'WSeed'})
df_lossSeeds = df_seeds.rename(columns={'TeamID':'LTeamID', 'seed_int':'LSeed'})
# df_seeds.head()

In [10]:
df_dummy = pd.merge(left=df_tour, right=df_lossSeeds, how='left', on=['Season', 'LTeamID'])
# df_dummy.head()

In [11]:
df_concat = pd.merge(left=df_dummy, right=df_winSeeds, how='left' ,on=['Season', 'WTeamID'])
# df_concat['SeedDiff'] = df_concat.WSeed - df_concat.LSeed
# df_concat.head()

Dataframe with seed, relative seed, and win/loss. Use for training

In [12]:
df_wins = pd.DataFrame()
df_wins['Seed'] = df_concat['WSeed']
# df_wins['SeedDiff'] = df_concat['SeedDiff']
df_wins['TeamID'] = df_concat['WTeamID']
df_wins['Season'] = df_concat['Season']
df_wins['Result'] = 1
df_wins = pd.merge(left=df_wins, right=df_rank, how='left', on=['Season', 'TeamID'])
# df_wins.head()

In [13]:
df_losses = pd.DataFrame()
df_losses['Seed'] = df_concat['LSeed']
# df_losses['SeedDiff'] = -df_concat['SeedDiff']
df_losses['TeamID'] = df_concat['LTeamID']
df_losses['Season'] = df_concat['Season']
df_losses['Result'] = 0
df_losses = pd.merge(left=df_losses, right=df_rank, how='left',on=['Season', 'TeamID'])
# df_losses.head()

In [14]:
df_lossesOpp = df_losses.copy()
df_lossesOpp.drop(labels=['Season', 'Result'], inplace=True, axis=1)
new_names = [(i,'Opp'+i) for i in df_lossesOpp.columns.values]
df_lossesOpp.rename(columns = dict(new_names), inplace=True)
# df_lossesOpp.head()

In [15]:
df_winsOpp = df_wins.copy()
df_winsOpp.drop(labels=['Season', 'Result'], inplace=True, axis=1)
new_names = [(i,'Opp'+i) for i in df_winsOpp.columns.values]
df_winsOpp.rename(columns = dict(new_names), inplace=True)
# df_winsOpp.head()

In [16]:
df_winloss = pd.concat([df_wins, df_lossesOpp], axis=1)

df_losswin = pd.concat([df_losses, df_winsOpp], axis=1)
# df_losswin.head()

In [17]:
df_finalData = pd.concat((df_winloss, df_losswin))
results = df_finalData['Result']
df_finalData.drop(labels=['Result'], inplace=True, axis=1)
df_finalData.insert(0, 'Result', results)
df_finalData.drop(labels=['TeamID', 'Season', 'OppTeamID'], inplace=True, axis=1)
# df_finalData.head()

In [18]:
df_finalData.isnull().any()

Result     False
Seed       False
MAS        False
SAG        False
POM        False
OppSeed    False
OppMAS     False
OppSAG     False
OppPOM     False
dtype: bool

## MLP Props
droput rate, batch size, epoch size

In [20]:
dropRate = 0.5
numBatch = 5
numEpoch = 50

### MLP Base 20 Nodes

In [31]:
# base 
base = Sequential()
base.name = 'base20'
base.add(Dense(5, input_dim=xDim, kernel_initializer='normal', activation = 'relu'))
base.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
# Compile base
base.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


### MLP Base 20 Nodes experimental

In [32]:
# base base20
base20 = Sequential()
base20.name = 'base20'
base20.add(Dense(20, input_dim=xDim, kernel_initializer='normal',activation = 'sigmoid'))
base20.add(Dropout(dropRate))
base20.add(BatchNormalization())
base20.add(Dense(20, input_dim=xDim, kernel_initializer='normal',activation = 'sigmoid'))
base20.add(Dropout(dropRate))
base20.add(BatchNormalization())
base20.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
# Compile base20
base20.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


# Fit the Model 

### Models of Interest

In [33]:
model = [base20]

###  StandardScaler


In [35]:
X = df_finalData.iloc[:,1:]
# Set input_dim
xDim = np.shape(X)[1]
X= X.values.reshape(-1,xDim)

y = df_finalData.Result.values

scaler = StandardScaler()
X = scaler.fit_transform(X)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, stratify=y)

print('Feature vector dimension is: %.2f' % xDim)
print(X[:5,:])

Feature vector dimension is: 8.00
[[ 1.86369018  4.39582579  3.78907735  3.87740966  1.86369018  3.48860575
   3.22990179  3.36310125]
 [-0.89148568 -0.61656495 -0.66504522 -0.59311733  1.43981697  1.31127764
   1.22458254  1.18718103]
 [-0.46761247 -0.57120395 -0.39509839 -0.31618203  1.01594376 -0.23099644
  -0.18299732 -0.15793328]
 [-0.04373926 -0.54852345 -0.6457633  -0.69202279  0.59207055  0.38137709
   0.10623142  0.13878311]
 [-1.31535889 -0.84336997 -0.80001863 -0.79092826  1.86369018  3.69273026
   3.11421029  2.73010627]]


/usr/local/lib/python3.5/dist-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


###  with scaled data

In [ ]:
# Fit the model
results=[]
# for i in range(0,np.size(model)):  
#     TB = TensorBoard(log_dir="logs/{}_{}{}".format(model[i].name, localtime().tm_hour, localtime().tm_min))
#     model[i].fit(X, y, epochs=numEpoch, batch_size=numBatch, verbose=0, callbacks = [TB])

#     kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
#     cvresults = cross_val_score(model[i], X, y, cv=kfold, scoring='neg_log_loss', verbose=0)
#     results.append(cvresults)
#     print("{}: {} ({})".format(model[i].__name__, results.mean(), results.std()))
TB = TensorBoard(log_dir="logs/{}_{}{}".format(base20.name, localtime().tm_hour, localtime().tm_min))
base20.fit(X, y, epochs=numEpoch, batch_size=numBatch, verbose=0, callbacks = [TB])

In [37]:
from sklearn.metrics import log_loss
y_pred =  base20.predict(X_test).reshape(-1,1)
LL = log_loss( y_test, y_pred)
print(LL)
# kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
# cvresults = cross_val_score(base20, X, y, cv=kfold, scoring='neg_log_loss', verbose=0)
# print('Log-Loss Mean :{:.3} ({:.3})'.format(np.mean(cvresults), np.std(cvresults)))

0.598174082733273


In [ ]:
# boxplot algorithm comparison
# fig = plt.figure()
# fig.suptitle('Algorithm Comparison')
# ax = fig.add_subplot(111)
# plt.boxplot((results))
# names = [str(i) for i in model]
# ax.set_xticklabels(names)
# plt.show()

In [ ]:
# df_sample_sub = pd.read_csv(data_dir + 'SampleSubmissionStage1.csv')
# n_test_games = len(df_sample_sub)

# def get_year_t1_t2(ID):
#     """Return a tuple with ints `year`, `team1` and `team2`."""
#     return (int(x) for x in ID.split('_'))

In [ ]:
# X_test = np.zeros(shape=(n_test_games, xDim))
# for ii, row in df_sample_sub.iterrows():
#     year, t1, t2 = get_year_t1_t2(row.ID)
#     t1_seed = df_seeds[(df_seeds.TeamID == t1) & (df_seeds.Season == year)].seed_int.values[0]
#     t2_seed = df_seeds[(df_seeds.TeamID == t2) & (df_seeds.Season == year)].seed_int.values[0]
#     diff_seed = t1_seed - t2_seed
#     X_test[ii, 0] = t1_seed
#     X_test[ii, 1] = diff_seed
#     X_test[ii, 2] = t1
#     X_test[ii, 3] = year


### Scale data with appropriate scaler

In [ ]:
# X_test = scaler.fit_transform(X_test)

## Make Predictions ##
Create predictions using the logistic regression model we trained.

In [ ]:
# preds = base2.predict(X_test)

In [ ]:
# clipped_preds = np.clip(preds, 0.05, 0.95)
# df_sample_sub.Pred = clipped_preds
# df_sample_sub.head()

Lastly, create your submission file!

In [ ]:
# filename = 'KerasMLP_RankSeedsYear'
# c=0
# ext = '.csv'
# if os.path.exists(filename+ext):
#     while os.path.exists(filename+ext):
#         c+=1
#         filename = filename+'_'+str(c)
#     df_sample_sub.to_csv(filename+ext, index=False)
# else:
#     df_sample_sub.to_csv(filename+ext, index=False)